# Getting started

*ironArray* for Python is a package that implements a multi-dimensional, compressed data container and an optimized computational engine to manage large arrays.

In this tutorial we will cover creating a simple ironArray array.  We will instatiate a simple array, then set properties on the array object.  We will also see how to set default properties by changing global and contextual configuration settings.


## Creating an array

Let's start by creating a simple array:

In [1]:
import numpy as np
import iarray as ia

dtshape = ia.DTShape([5, 5], np.float64)
arr = ia.linspace(dtshape, -1 , 1)
print(repr(arr))

<IArray (5, 5) np.float64>


Voilà, the object `arr` contains our first ironArray array.

To create an array, we first have to define its shape and data type, using the `DTShape` dataclass.  The array is then instantiated by the `linspace` constructor, where you specify the `start` and `stop` values.  Functions in ironArray are written to map closely to NumPy functions;  you can consult the [NumPy documentation](https://numpy.org/doc/) for more information on the functions and their parameters.

The ironArray library is designed to operate on floating point numerical data.  Consequently, the arrays currently support two data types: `double` and `float`.

Let's convert the `arr` object into a NumPy array and inspect the data:

In [2]:
ia.iarray2numpy(arr)

array([[-1.        , -0.91666667, -0.83333333, -0.75      , -0.66666667],
       [-0.58333333, -0.5       , -0.41666667, -0.33333333, -0.25      ],
       [-0.16666667, -0.08333333,  0.        ,  0.08333333,  0.16666667],
       [ 0.25      ,  0.33333333,  0.41666667,  0.5       ,  0.58333333],
       [ 0.66666667,  0.75      ,  0.83333333,  0.91666667,  1.        ]])

### Properties

Besides the shape and data type, we can set more properties on the array.  For example, let's make it persistent:

In [3]:
pers_arr = ia.linspace(dtshape, -1 , 1, urlpath="myarr.iarr")

In [4]:
%%bash
ls -l myarr.iarr

-rw-r--r-- 1 faltet faltet 606 Jan 20 11:02 myarr.iarr


and then we'll read the persistent object from disk.  We are going to use `ia.open()`instead of `ia.load()` to lazily read in the data as needed (a topic covered in a later turorial):

In [5]:
arr2 = ia.open("myarr.iarr")
print(arr2.data)

[[-1.         -0.91666667 -0.83333333 -0.75       -0.66666667]
 [-0.58333333 -0.5        -0.41666667 -0.33333333 -0.25      ]
 [-0.16666667 -0.08333333  0.          0.08333333  0.16666667]
 [ 0.25        0.33333333  0.41666667  0.5         0.58333333]
 [ 0.66666667  0.75        0.83333333  0.91666667  1.        ]]


#### Storage

The `Storage` class is used to tune the storage for your arrays.  The `urlpath` property is just one of many properties that can be set in a `Storage` object.  See the [Storage documentation](../reference/autofiles/config/iarray.Storage.html) for more details on how ironArray storage can be optimized to improve performance and decrease array storage size.

In [6]:
store = ia.Storage()
print(store)

Storage(chunkshape=None, blockshape=None, urlpath=None, enforce_frame=False, plainbuffer=False)


We can also set multiple properties in a single `Storage` instance. For example, this `Storage` object has properties for the shape of the chunks and the blocks:

```
ia.Storage(chunkshape=(3000, 1000), blockshape=(100, 100))
```

The following example shows how to create a `Storage` object and set its properties, then add it to a larger ironArray array object:

In [7]:
dtshape = ia.DTShape((10000, 7000), np.float64)
storage = ia.Storage(chunkshape=(3000, 1000), blockshape=(100, 100), urlpath="large_arr.iarr")
arr = ia.linspace(dtshape, -1, 1, storage=storage)

In [8]:
%%bash
ls -lh large_arr.iarr

-rw-r--r-- 1 faltet faltet 103M Jan 20 11:02 large_arr.iarr


We have just created an array containing about 500MB of data.  Thanks to integrated compression, the size of the serialized array on disk is only 100MB, about 5 times smaller.  In contrast to ordinary chunked and compressed data container libraries that support just a single level of data partitioning (such as HDF5 and Zarr), IronArray allows for two levels: chunks and blocks.  As we'll see later, two levels offer more flexibility and options for tuning performance on modern CPU architectures.

#### More Properties

You may set many other properties when creating an ironArray array.  Here we set some compression properties:

In [9]:
dtshape = ia.DTShape((10000, 7000), np.float64)
storage = ia.Storage(chunkshape=(3000, 1000), blockshape=(100, 100), urlpath="large_arr2.iarr")
arr = ia.linspace(dtshape, -1, 1, storage=storage, clevel=1, codec=ia.Codecs.ZSTD, fp_mantissa_bits=30)

In [10]:
%%bash
ls -lh large_arr2.iarr

-rw-r--r-- 1 faltet faltet 31M Jan 20 11:02 large_arr2.iarr


As you can see, we created an array that holds 500MB of data, as before.  But now the serialized data only occupies 30MB of disk space.  We changed the compression codec and mantissa bits properties to shrink the storage size:

1) `codec=ia.Codecs.ZSTD`:  ZSTD offers better compression.

2) `fp_mantissa_bits=30`:  We are setting just 30 significant bits in the mantissa, improving the compression ratio. You can set the `fp_mantissa_bits` to any precision between 1 and 24 bit (float32) or 53 bit (float64); the compression engine will compress the data to fit the specified precision.

You can see the complete set of supported properties and their defaults by examining an instance of `ia.Config`:

In [11]:
cfg = ia.Config()
print(cfg)

Config(codec=<Codecs.LZ4: 1>, clevel=5, filters=[<Filters.SHUFFLE: 1>], fp_mantissa_bits=0, use_dict=False, nthreads=28, eval_method=<Eval.AUTO: 1>, seed=1, random_gen=<RandomGen.MERSENNE_TWISTER: 0>, storage=Storage(chunkshape=None, blockshape=None, urlpath=None, enforce_frame=False, plainbuffer=False), chunkshape=None, blockshape=None, urlpath=None, enforce_frame=False, plainbuffer=False)


### Configuration

Setting the same compression and storage-related properties every time you create an ironArray array object can become tedious and repetitive, if you are dealing with known datasets with stable characteristics.  You have the option to set default properties in either the global configuration or within a context.

#### Global configuration

If you will always use the same configuration parameters in your script, it might be a good idea to set default global properties as part of your script initialization:

In [12]:
ia.set_config(codec=ia.Codecs.ZSTD, clevel=1)

Config(codec=<Codecs.ZSTD: 5>, clevel=1, filters=[<Filters.SHUFFLE: 1>], fp_mantissa_bits=0, use_dict=False, nthreads=28, eval_method=<Eval.AUTO: 1>, seed=1, random_gen=<RandomGen.MERSENNE_TWISTER: 0>, storage=Storage(chunkshape=None, blockshape=None, urlpath=None, enforce_frame=False, plainbuffer=False), chunkshape=None, blockshape=None, urlpath=None, enforce_frame=False, plainbuffer=False)

You can verify that the new default properties are now set:  the default compression codec has changed to ZSTD, and the default compression level has changed to 1.

In [13]:
cfg = ia.Config()
print(cfg)

Config(codec=<Codecs.ZSTD: 5>, clevel=1, filters=[<Filters.SHUFFLE: 1>], fp_mantissa_bits=0, use_dict=False, nthreads=28, eval_method=<Eval.AUTO: 1>, seed=1, random_gen=<RandomGen.MERSENNE_TWISTER: 0>, storage=Storage(chunkshape=None, blockshape=None, urlpath=None, enforce_frame=False, plainbuffer=False), chunkshape=None, blockshape=None, urlpath=None, enforce_frame=False, plainbuffer=False)


These will be the defaults for *all* the ironArray functions that are called in your script.

#### Contextual Configuration

Sometimes you want different configuration profiles for different kinds of arrays.  In this case, you can create `ia.config` objects with custom settings that can be applied to selected arrays.  This is an example of *contextual configuration*:

In [14]:
dtshape = ia.DTShape([1000, 1000])
with ia.config(clevel=9, codec=ia.Codecs.LZ4) as cfg:
    a1 = ia.linspace(dtshape, -1, 0, cfg=cfg)
a2 = ia.linspace(dtshape, -1, 0)
print(f"a1 cratio: {a1.cratio}")
print(f"a2 cratio: {a2.cratio}")

a1 cratio: 8.781161022342813
a2 cratio: 15.030573209604679


In this case, `a1` and `a2` have different compression ratios, as they have different compression levels and compression codecs set as default properties on their array configurations.  `a1` is using the LZ4 codec with compression level 3, whereas `a2` is using ZSTD and compression level 1, the global defaults that we set in the previous example.

### Conclusion

Now you can create ironArray arrays with specific properties.  You can create arbitrarily large arrays either in memory or on disk, and you can tailor arrays to your own needs using ironArray configuration properties.  Use the advanced global and contextual configurations to set often-used configuration profiles for your arrays.